## Яндекс Практикум, курс "Инженер Машинного Обучения" (2024 г.)
## Проект 4-го спринта: "Создание рекомендательной системы"

## Этап 3. Оффлайн-рекомендации

### Постановка задачи

Подготовьте данные для обучения:
- Загрузите ранее сохранённые данные.
- Разбейте данные на тренировочную и тестовую выборки: в тренировочную отнесите все данные до __16 декабря 2022 г. (не включительно)__.

Рассчитайте рекомендации нескольких типов, сохраните каждый тип в своём файле:
- топ популярных — в `top_popular.parquet`,
- персональные (при помощи ALS) — в `personal_als.parquet`,
- похожие треки или i2i (при помощи ALS) — в `similar.parquet`.

Постройте ранжирующую модель, которая использует минимум три признака. Отранжируйте рекомендации, а итоговые сохраните в `recommendations.parquet`.

Проверьте оценку качества трёх типов рекомендаций: 
1. топ популярных,
2. персональных, полученных при помощи ALS,
3. итоговых (по метрикам: recall, precision, coverage, novelty).

Приведите значения метрик в README.

### Инициализация

In [1]:
import os
import numpy as np
import pandas as pd
import joblib
import ipywidgets

from sklearn.metrics import accuracy_score

In [2]:
import sys

# Проверяем, в каком окружении работаем
print(sys.executable)

c:\Users\Kirill_Nosov\_Repos\.venv\Scripts\python.exe


In [2]:
# Отключаем предупреждения
import warnings
from pandas.errors import SettingWithCopyWarning

warnings.simplefilter(action='ignore', category=(SettingWithCopyWarning, FutureWarning))

### Загрузка и разбиение данных

In [4]:
# Загружаем основные данные
items = pd.read_parquet("items.parquet")
events = pd.read_parquet("events.parquet")

Разбиваем данные на тренировочную и тестовую выборки (в последнюю относим данные до 16 декабря 2022 г. не включительно).

In [5]:
# Зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2022-12-16")

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

### Рекомендации на основе самых популярных треков

Загружаем ранее сохраненные топ-треки и оцениваем качество рекомендаций на их основе для "холодных" пользователей.

In [7]:
# Загружаем топ-100 самых прослушиваемых треков
top_k_pop_items = pd.read_parquet("top_popular.parquet")
top_k_pop_items.head()

,item_id,plays,users,name,genres,artists,albums
0,53404,111062,111062,Smells Like Teen Spirit,"[allrock, alternative, rock]",[Nirvana],"[Nevermind, Smells Like Teen Spirit / In Bloom..."
1,33311009,106921,106921,Believer,"[allrock, rock]",[Imagine Dragons],"[Best Party Hits, Graduation 2021, Hello Summe..."
2,178529,101924,101924,Numb,"[metal, numetal]",[Linkin Park],"[00s Rock Anthems, Meteora]"
3,35505245,99490,99490,I Got Love,"[rap, rusrap]","[Рем Дигга, Miyagi & Эндшпиль]",[I Got Love]
4,65851540,86670,86670,Юность,"[pop, ruspop]",[Dabro],[Юность]


In [6]:
# Идентификаторы уникальных пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()

In [9]:
# Идентификаторы "холодных" пользователей
cold_users = users_test[~users_test.isin(users_train)]

In [10]:
# Формируем бинарные колонки target и prediction

top_k_pop_items['prediction'] = 1

cold_users_events = \
    events_test[events_test["user_id"].isin(cold_users)] \
        .merge(top_k_pop_items[['item_id', 'prediction']], on="item_id", how="left")

cold_users_events['prediction'].fillna(0, inplace=True)

cold_users_events['target'] = 1

In [11]:
# Точность рекомендаций
accuracy = accuracy_score(cold_users_events['prediction'], cold_users_events['target'])
print(accuracy)

0.05650518569721842


Точность получилась не очень высокой. Отчасти, это может быть связано с общим снижением кол-ва прослушиваний и пользователей после ноября 2022 г., т.е. могло измениться распределение данных (см. EDA).

Посчитаем покрытие "холодных" пользователей рекомендациями.

In [12]:
cold_users_hit_ratio = cold_users_events.groupby("user_id").agg(hits=("prediction", 'sum'))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: 0.42
Среднее покрытие пользователей: 3.53


Таким образом, рекомендациями на основе топ-треков охвачено чуть менее половины "холодных" пользователей. 
При этом каждый из охваченных пользователей получил в среднем 3.5 трека.

### Персональные рекомендации на основе коллаборативного подхода и ALS

[Техническая документация на библиотеку implicit](https://benfred.github.io/implicit/api/models/cpu/als.html)


Перекодируем идентификаторы `user_id` и `item_id`.

In [6]:
import scipy
import sklearn.preprocessing

# Перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# Перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train = events_train.merge(items[["item_id", "item_id_enc"]], on="item_id", how='left') 
events_test = events_test.merge(items[["item_id", "item_id_enc"]], on="item_id", how='left') 

In [14]:
# Сократим размер типа для экономии памяти 
items['item_id_enc'] = items['item_id_enc'].astype('int32')
events_train[['item_id_enc', 'user_id_enc']] = events_train[['item_id_enc', 'user_id_enc']].astype('int32')
events_test[['item_id_enc', 'user_id_enc']] = events_test[['item_id_enc', 'user_id_enc']].astype('int32')

Создаем sparse-матрицу в формате [CSR](https://docs.scipy.org/doc/scipy/reference/sparse.html)

In [17]:
# Формируем колонку target (1 - трек прослушан, 0 - нет)
events_train['target'] = 1

# Создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["target"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

Имея подготовленную матрицу взаимодействий, создаем ALS-модель. Для примера возьмём количество латентных факторов для матриц $P и Q$ равным 50. 

In [18]:
# Код для создания и тренировки модели
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

c:\Users\Kirill_Nosov\_Repos\.venv\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/50 [00:00<?, ?it/s]

In [12]:
# Сохраняем als-модель в файл
with open('models/als_model.pkl', 'wb') as fd:
    joblib.dump(als_model, fd)

Чтобы получить рекомендации для отдельного пользователя с помощью модели ALS, используем функцию ниже.

In [19]:
def get_recommendations_als(user_item_matrix, als_model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = als_model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

Посмотрим историю и рекомендации для случайного пользователя.

In [ ]:
# Выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["name", "genres", "albums", "artists"]], on="item_id")
)
user_history_to_print = user_history[["item_id", "name", "genres", "albums", "artists"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_als(user_item_matrix_train, als_model, user_id, user_encoder, item_encoder, include_seen=True, n=10)
user_recommendations = user_recommendations.merge(items.set_index("item_id")[["name", "genres", "albums", "artists"]], on="item_id")
user_recommendations_to_print = \
    user_recommendations[["item_id", "name", "genres", "albums", "artists", "score"]].head(10)
display(user_recommendations)

TODO: Проанализируйте, релевантны ли рекомендации имеющейся истории.

Получим рекомендации для всех имеющихся пользователей.

In [20]:
# Получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# Получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, 
    N=100)

Код возвращает рекомендации как список списков, что не очень удобно. Преобразуем его в более удобный табличный формат.

In [21]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [37]:
# Сохраним полученные рекомендации в файл
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("personal_als.parquet")

Оценим точность рекомендаций на тестовой выборке с порогом вероятности 0.5.

In [22]:
# Формируем бинарные колонки target и prediction, для последней берем порог вероятности 0.5

events_test['target'] = 1

als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "target"]], on=["user_id", "item_id"], how="left")
)

als_recommendations['target'] = als_recommendations['target'].fillna(0)

als_recommendations['prediction'] = (als_recommendations['score'] >= 0.5).astype('int')

In [ ]:
# Точность рекомендаций
accuracy = accuracy_score(als_recommendations['prediction'], als_recommendations['target'])
print(accuracy)

TODO: вывод

Посчитаем несколько специфичных метрик. 

Найдем покрытие по объектам, используя все имеющиеся рекомендации.

In [ ]:
cov_items = als_recommendations['item_id'].nunique() / len(items) 
print(cov_items)

TODO: Вывод

Посчитаем среднее Novelty@5, для этого:
- разметим каждую рекомендацию в als_recommendations булевым признаком played, используя events_train
(False — пользователь не прослушал трек, True — прослушал),
- посчитаем Novelty@5 для каждого пользователя,
- найдем среднее значение полученных Novelty@5.

In [ ]:
# разметим каждую рекомендацию признаком played
events_train["played"] = True
als_recommendations = als_recommendations \
    .merge(events_train[['played', "user_id", "item_id"]], on=["user_id", "item_id"], how="left")
als_recommendations["played"] = als_recommendations["played"].fillna(False).astype("bool")

# проставим ранги
als_recommendations = als_recommendations.sort_values(by='score', ascending=False) 
als_recommendations["rank"] = als_recommendations.groupby("user_id").cumcount() + 1

# посчитаем novelty по пользователям
novelty_5 = (1 - als_recommendations.query("rank <= 5").groupby("user_id")["played"].mean())

# посчитаем средний novelty
novelty_5_mean = novelty_5.mean()
print(novelty_5_mean)

TODO: вывод

Посчитаем recall и precision для ALS-рекомендаций. Для этого события в тестовой выборке и рекомендации для одних и тех же пользователей разметим признаками:
- gt (ground truth): объект есть в тестовой выборке;
- pr (predicted): объект есть в рекомендациях.

Теперь разметим признаки бинарной классификации:
- TP: объект есть и в тестовой выборке, и в рекомендациях (истинная рекомендация),
- FP: объекта нет в тестовой выборке, но он есть в рекомендациях (ложноположительная рекомендация),
- FN: объект есть в тестовой выборке, но его нет в рекомендациях (ложноотрицательная рекомендация)

Создаем функцию для подсчета метрик TP, FP и FN у пользователей, которые есть одновременно и в events_train,
и в events_test.

In [ ]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

Считаем TP, FP и FN для каждой пары user_id, item_id

In [ ]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(events_train,
                                                                        events_test, 
                                                                        als_recommendations,
                                                                        top_k=5)

Считаем precision и recall

In [ ]:
def compute_cls_metrics(events_recs_for_binary_metrics):
    
    groupper = events_recs_for_binary_metrics.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum() / (groupper["tp"].sum() + groupper["fn"].sum()) # ваш код здесь
    recall = recall.fillna(0).mean()

    return precision, recall

In [ ]:
precision, recall = compute_cls_metrics(events_recs_for_binary_metrics)
print(round(precision, 3), round(recall, 3))

### Рекомендации на основе контентного подхода и ALS

Построим контентные рекомендации на основе принадлежности треков к различным жанрам.

Сначала загрузим список жанров по всем трекам, который мы создали на предыдущем этапе.

In [ ]:
genres = pd.read_parquet('genres.parquet')
genres.head(5)

Функция ниже строит матрицу вида «трек-жанр»

In [ ]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genres"] is None: # После предобработки NA быть не может, оставили для общего случая
            continue
        
        item_genres = list(v["genres"])
        
        for genre_name in item_genres:
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(1)
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), \
                                         shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанрам была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

Получим матрицу с весами по жанрам для каждого трека:

In [ ]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

Будем искать рекомендации на основе косинусной близости. Функция ниже находит рекомендации для одного пользователя

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity


def get_recommendations_by_genres(user_id, items, events, all_items_genres_csr, n=5):

    """ Возвращает n рекомендаций для user_id на основе предпочитаемых им жанров"""
    
    user_events = events.query("user_id == @user_id")
    user_items = items[items["item_id"].isin(user_events["item_id"])]

    user_items_genres_csr = all_items_genres_csr[user_items['item_id_enc']]
    
    # вычислим среднюю склонность пользователя к жанрам 
    user_genres_scores = np.asarray(user_items_genres_csr.mean(axis=0))

    # вычисляем сходство между треками пользователя и всеми имеющимися трекам
    similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

    # преобразуем в одномерный массив
    similarity_scores = similarity_scores.flatten()

    # получаем индексы top-n (по убыванию значений), т.е. encoded индексы самых похожих треков
    top_n_indices = np.argsort(-similarity_scores)[:n] 

    selected_items = items[items["item_id_enc"].isin(top_n_indices)]
    
    return selected_items['item_id'].to_list(), similarity_scores[top_n_indices].tolist()

Посмотрим рекомендации для произвольного пользователя

In [ ]:
# Выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["name", "genres", "albums", "artists"]], on="item_id")
)
user_history_to_print = user_history[["item_id", "name", "genres", "albums", "artists"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
selected_items, similarity_scores = \
    get_recommendations_by_genres(user_id, items, events_train, all_items_genres_csr, n=5)
selected_items = items[items["item_id"].isin(selected_items)]
selected_items['scores'] = similarity_scores

with pd.option_context("max_colwidth", 100):
   display(selected_items[[["item_id", "name", "genres", "albums", "artists", 'scores']]])

TODO: Вывод

Запускаем поиск для всех пользователей из events_train

In [ ]:
# Для примера взял только первые 1000 юзеров

users_ids = events_train['user_id'].unique()[:1000] 

n = 5
items_ids = []
scores = []
for i, user_id in enumerate(users_ids):
    user_items_ids, user_scores = get_recommendations_by_genres(user_id, items, events_train, genres, all_items_genres_csr, n)
    items_ids.append(user_items_ids)
    scores.append(user_scores)

    if i >= 1000:
        break

content_recommendations = pd.DataFrame({
    "user_id": users_ids,
    "item_id": items_ids, 
    "score": scores})

content_recommendations = content_recommendations.explode(["item_id", "score"], ignore_index=True)

In [ ]:
content_recommendations

In [ ]:
# Сохраним полученные рекомендации в файл
content_recommendations.to_parquet("content_recommendations.parquet")

### Рекомендации на основе похожих треков (i2i) и ALS

Получим рекомендации при помощи метода `similar_items` ([описание](https://benfred.github.io/implicit/api/models/cpu/als.html)).

In [ ]:
# Загружаем ALS-модель
with open('models/als_model.pkl', 'rb') as fd:
    als_model = joblib.load(fd)

In [ ]:
# получим энкодированные идентификаторы всех объектов в events_train
train_item_ids_enc = events_train['item_id_enc'].unique()

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(), 
    "score": sim_item_scores.tolist()}) 
similar_items = similar_items.explode(['sim_item_id_enc', 'score'], ignore_index=True) 

# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype('int') 
similar_items["score"] = similar_items["score"].astype("float")

# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"]) 
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"]) 
similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

In [ ]:
# Сохраним полученные рекомендации в файл
similar_items.to_parquet("similar.parquet")

С помощью функции ниже можно посмотреть рекомендации для одного трека

In [ ]:
def print_sim_items(item_id, similar_items):
    item_columns_to_use = ["item_id", "name", "genres", "artists", "albums"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(items[item_columns_to_use].set_index("item_id"), left_on="item_id_2", right_index=True)
    display(si)

In [ ]:
# Задаем произвольный трек
item_id = events_train['item_id'].sample().iat[0]

# Смотрим похожие треки
print_sim_items(item_id, similar_items)

TODO: Проанализируйте адекватность результатов.

### Построение ранжирующей модели

Подготовим список кандидатов для обучения ранжирующей модели. В качестве кандидатогенераторов возьмём ранее подготовленные при помощи ALS коллаборативные и контентные рекомендации. Кроме того, добавим 3-й признак - кол-во треков, прослушанных каждым пользователем.

In [39]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("personal_als.parquet")
content_recommendations = pd.read_parquet("content_recommendations.parquet")

In [ ]:
# Объединяем списки рекомендаций по совпадению user_id, item_id
candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer") 

Добавим в candidates таргеты, используя историю взаимодействий в events_train:
- 1 - для прослушанных item_id (положительный пример),
- 0 - для всех остальных (негативный пример).

В candidates отберем все положительные примеры, а также не менее четырёх негативных для каждого пользователя в положительных примерах.

In [42]:
# Добавляем таргет к кандидатам
events_train["target"] = 1
candidates = candidates.merge(events_train[["user_id", "item_id", "target"]], 
                              on=["user_id", "item_id"], 
                              how='left') 
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

# для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"), 
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
    ])

Добавим в events_train новый признак пользователей - количество треков, прослушанных каждым пользователем.

In [ ]:
def get_user_features(events):
    """ считает пользовательские признаки """
    
    user_features = events.groupby("user_id").agg(
        tracks_played=("started_at", "count")
    )

    return user_features
    

user_features_for_train = get_user_features(events_train)
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")

Обучаем модель

In [29]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score', 'tracks_played']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6526062	total: 175ms	remaining: 2m 54s
100:	learn: 0.5119048	total: 2.6s	remaining: 23.1s
200:	learn: 0.5112058	total: 5.07s	remaining: 20.2s
300:	learn: 0.5105778	total: 7.59s	remaining: 17.6s
400:	learn: 0.5100299	total: 10.1s	remaining: 15.1s
500:	learn: 0.5095632	total: 12.5s	remaining: 12.4s
600:	learn: 0.5091520	total: 14.9s	remaining: 9.88s
700:	learn: 0.5087528	total: 17.3s	remaining: 7.36s
800:	learn: 0.5084194	total: 19.9s	remaining: 4.94s
900:	learn: 0.5080962	total: 22.6s	remaining: 2.48s
999:	learn: 0.5078292	total: 25.2s	remaining: 0us


In [30]:
# Сохраняем ранжирующую модель
os.makedirs('models/', exist_ok=True)
cb_model.save_model('models/cb_model.cbm')

Теперь можно получить финальные рекомендации, используя обученную модель.

In [ ]:
# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_for_train[candidates_for_train["user_id"].isin(events_test["user_id"].drop_duplicates())]

inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby('user_id').cumcount() + 1 

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank.query('rank <= @max_recommendations_per_user')

Посмотрим, сколько пользователей попало в final_recommendations

In [ ]:
final_recommendations['user_id'].nunique()

In [ ]:
# Сохраним последний вариант персональных рекомендаций в файл
final_recommendations.to_parquet("final_recommendations_feat.parquet")

Проведем валидацию финальных рекомендаций.

Посчитаем метрики recall и precision, используя отложенную тестовую выборку, 
а также функции process_events_recs_for_binary_metrics и compute_cls_metrics.

In [50]:
cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_train,
    events_test, 
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5) # ваш код здесь

Common users: 75194


In [51]:
print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

precision: 0.006, recall: 0.015


Посмотрим, какие признаки вносят наибольший вклад в ранжирование. 

In [ ]:
feature_importance = pd.DataFrame(cb_model.get_feature_importance(), 
    index=features, 
    columns=["fi"])

feature_importance = feature_importance.sort_values('fi', ascending=False)
feature_importance